# Tokenizer

* [Tiktokens](https://tiktokenizer.vercel.app/)
* [Hugging Face Tokenizer docs](https://huggingface.co/docs/transformers/v5.1.0/en/fast_tokenizers)
* [Stanford Assignment1](https://github.com/stanford-cs336/assignment1-basics/blob/main/cs336_spring2025_assignment1_basics.pdf)


## Summary

#### Tokenizer: Strings <-> Tokens (indices)

Tokenization is a foundational NLP step that breaks text into smaller units (tokens) so machine learning models can process it. <br>
Why it matters?: A Transformers model expects the input to be a PyTorch or NumPy tensor. strings -> tokens -> tensors -> transformers


```

input strings: Hello, This is CiNet, NICT.
output tokens: 13225, 11, 1328, 382, 21572, 9944, 11, 478, 28594, 13

```
#### Methods
* Word-Based Tokenization
* Character-Based Tokenization
* Byte-Based Tokenization

##### 1. Word-Based Tokenization

This method splits text using delimiters(definition) such as spaces and punctuation.

**Example**

```
"Tokenization is great"
→ ["Tokenization", "is", "great"]
```
* ✅ Pros
    * Intuitive and human-readable
    * Each token usually carries semantic meaning

* ❌ Cons
    * **Enormous vocabulary**: Every unique word must be stored, often resulting in hundreds of thousands of tokens
    * **Out-of-Vocabulary (OOV) issues**: Cannot handle unseen words (e.g., typos, slang, new words)
    * **Data sparsity**: Related words like *run* and *running* are treated as completely different tokens

---

#### 2. Character-Based Tokenization

This method splits text into individual characters (letters, digits, punctuation, etc.).

**Example**

```
"Token"
→ ["T", "o", "k", "e", "n"]
```

* ✅ Pros
    * **Very small vocabulary**: About 256 possible characters in basic English
    * **No OOV problem**: Any word can be constructed from characters

* ❌ Cons
    * **Very long sequences**: Leads to high computational cost (especially for Transformers)
    * **Low semantic meaning**: Individual characters carry little meaning on their own

#### 3. Byte-Based (Byte-Level) Tokenization

This method treats text as raw UTF-8 bytes instead of characters. Even complex characters (e.g., emojis or foreign scripts) are decomposed into byte sequences.

**Example**

* A 4-byte emoji is split into 4 individual bytes.

* ✅ Pros
    * **Language-agnostic**: Works well for multilingual text
    * **Robust to rare characters and emojis**
    * **No OOV problem**

* ❌ Cons
    * Can produce long token sequences
    * Often requires additional compression (e.g., BPE) for efficiency

---

#### Summary Comparison

| Feature          | Word-Based   | Character-Based   | Byte-Based        |
| ---------------- | ------------ | ----------------- | ----------------- |
| Granularity      | High (Words) | Low (Characters)  | Very Low (Bytes)  |
| Vocabulary Size  | Very Large   | Very Small (~256) | Very Small (~256) |
| OOV Handling     | Poor         | Excellent         | Excellent         |
| Sequence Length  | Short        | Very Long         | Very Long         |
| Semantic Meaning | High         | Low               | Low               |



## Bite-Pair Encoding

### BPE Encoding (Training & Encoding)

Suppose:

**Input string**

`"the cat ate the rat"`

**Initial Vocabulary**

`{0: b' ', 1: b'a', 2: b'c', 3: b'e', 4: b'h', 5: b't', 6: b'r'}`

---

#### Step 1 — Pre-tokenization

The input is split into space-aware chunks:

`"the cat ate the rat"  →  ['the', ' cat', ' ate', ' the', ' rat']`

Notice:

* Spaces stay attached to the following word.
* This is typical for GPT-style tokenizers.

---

#### Step 2 — Apply BPE Training (Learning Merges)

We now simulate **training**, where we repeatedly:

1. Count all adjacent token pairs
2. Merge the most frequent pair
3. Update the corpus
4. Repeat

---

##### 🔹 Iteration 1 — Count Adjacent Pairs

From the full corpus, we count all adjacent pairs:

| Pair   | Frequency |
| ------ | --------- |
| (a, t) | 3         |
| (t, h) | 2         |
| (h, e) | 2         |
| (c, a) | 1         |
| (r, a) | 1         |
| (t, e) | 1         |
| ( , c) | 1         |
| ( , a) | 1         |
| ( , t) | 1         |
| ( , r) | 1         |

* 🏆 Most Frequent Pair: `(a, t) → frequency 3`

---

**🔹 Merge Rule #1**

`a + t → at`

**Updated Corpus**

`["t","h","e"], [" ","c","at"], [" ","at","e"], ["t","h","e"], [" ","r","at"]`

**Updated Vocabulary**

`{0: b' ', 1: b'a', 2: b'c', 3: b'e', 4: b'h', 5: b't', 6: b'r', 7: b'at'}`

---

##### 🔹 Iteration 2 — Recalculate Frequencies

Now count again:

| Pair    | Frequency |
| ------- | --------- |
| (t, h)  | 2         |
| (h, e)  | 2         |
| ( , at) | 1         |
| (c, at) | 1         |
| (r, at) | 1         |
| (at, e) | 1         |
| ( , c)  | 1         |
| ( , r)  | 1         |

* 🏆 Most Frequent Pair: `(t, h) → frequency 2`

---

**🔹 Merge Rule #2**

`t + h → th`

**Updated Corpus**

`["th","e"], [" ","c","at"], [" ","at","e"], ["th","e"], [" ","r","at"]`

**Updated Vocabulary**

`{0: b' ', 1: b'a', 2: b'c', 3: b'e', 4: b'h', 5: b't', 6: b'r', 7: b'at', 8: b'th'}`

---

##### 🔹 Iteration 3 — Recalculate Frequencies

Now count again:

| Pair    | Frequency |
| ------- | --------- |
| (th, e) | 2         |
| ( , at) | 1         |
| (c, at) | 1         |
| (r, at) | 1         |
| (at, e) | 1         |
| ( , c)  | 1         |
| ( , r)  | 1         |

* 🏆 Most Frequent Pair: `(th, e) → frequency 2`

---

**🔹 Merge Rule #3**

`th + e → the`

**Updated Corpus**

`["the"], [" ","c","at"], [" ","at","e"], ["the"], [" ","r","at"]`

**Updated Vocabulary**

`{0: b' ', 1: b'a', 2: b'c', 3: b'e', 4: b'h', 5: b't', 6: b'r', 7: b'at', 8: b'th', 9: b'the'}`

---

##### 🔹 Stop Condition

Now the highest frequency is 1 for all pairs.
Training usually continues until:

* A target vocabulary size is reached
* Or no pair appears more than once

For this example, we stop here.

---

**Final Learned Merges**

1. (a, t) → at
2. (t, h) → th
3. (th, e) → the

---

**Final Vocabulary**

`{0: b' ', 1: b'a', 2: b'c', 3: b'e', 4: b'h', 5: b't', 6: b'r', 7: b'at', 8: b'th', 9: b'the'}`

---

**Now Encoding the Sentence**

Using the learned merges:

`['the', ' cat', ' ate', ' the', ' rat']`

Becomes:

`[b'the'], [b' ', b'c', b'at'], [b' ', b'at', b'e'], [b'the'], [b' ', b'r', b'at']`

Mapped to IDs:

`[9, 0, 2, 7, 0, 7, 3, 9, 0, 6, 7]`

---

##### 🔎 What This Shows

* BPE builds frequent subwords automatically.
* "the" becomes a single token.
* "at" becomes reusable for cat / ate / rat.
* Rare patterns stay broken into smaller units.

This is why BPE balances:

* Efficiency
* Small vocabulary
* No OOV problem


#### Training Code

In [ ]:
import regex as re
from collections import Counter, defaultdict
from typing import Dict, List, Tuple

PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""

def train_bpe(
    input_path: str,
    vocab_size: int,
    special_tokens: List[str],
) -> Tuple[Dict[int, bytes], List[Tuple[bytes, bytes]]]:

    # 1. Initialize Vocab
    vocab: Dict[int, bytes] = {i: bytes([i]) for i in range(256)}
    next_id = 256

    special_token_bytes = [s.encode("utf-8") for s in special_tokens]
    for st in special_token_bytes:
        if st not in vocab.values():
            vocab[next_id] = st
            next_id += 1

    # 2. Build Word Frequencies
    with open(input_path, "rb") as f:
        raw_bytes = f.read()

    # Split around special tokens
    segments = [raw_bytes]
    for st in special_token_bytes:
        new_segments = []
        for seg in segments:
            parts = seg.split(st)
            for i, part in enumerate(parts):
                if part: new_segments.append(part)
                if i < len(parts) - 1: new_segments.append(st)
        segments = new_segments

    word_freqs = Counter()
    for seg in segments:
        if seg in special_token_bytes:
            word_freqs[(seg,)] += 1
        else:
            # Decode to run regex, but keep it as a sequence of bytes
            decoded = seg.decode("utf-8", errors="ignore")
            for chunk in re.findall(PAT, decoded):
                # Represent word as a list of individual byte-tokens
                word = tuple(bytes([b]) for b in chunk.encode("utf-8"))
                word_freqs[word] += 1

    # 3. Efficient State Tracking
    unique_words = [list(w) for w in word_freqs.keys()]
    counts = list(word_freqs.values())
    
    pair_counts = defaultdict(int)
    pair_to_word_indices = defaultdict(set)

    for i, word in enumerate(unique_words):
        for pair in zip(word, word[1:]):
            pair_counts[pair] += counts[i]
            pair_to_word_indices[pair].add(i)

    merges: List[Tuple[bytes, bytes]] = []

    # 4. Main Merge Loop
    while len(vocab) < vocab_size:
        if not pair_counts:
            break
        
        best_pair = max(pair_counts.items(), key=lambda x: (x[1], x[0]))[0]
        if pair_counts[best_pair] <= 0:
            break

        merges.append(best_pair)
        new_token = best_pair[0] + best_pair[1]
        vocab[next_id] = new_token
        next_id += 1

        affected_word_indices = pair_to_word_indices.pop(best_pair)
        pair_counts.pop(best_pair)

        for i in affected_word_indices:
            word = unique_words[i]
            freq = counts[i]
            
            j = 0
            new_word = []
            while j < len(word):
                if j < len(word) - 1 and (word[j], word[j+1]) == best_pair:
                    
                    if j > 0:
                        prev_pair = (word[j-1], word[j])
                        pair_counts[prev_pair] -= freq
                    if j < len(word) - 2:
                        next_pair = (word[j+1], word[j+2])
                        pair_counts[next_pair] -= freq
                    
                    new_word.append(new_token)
                    j += 2
                else:
                    new_word.append(word[j])
                    j += 1
            
            unique_words[i] = new_word
            for j in range(len(new_word) - 1):
                pair = (new_word[j], new_word[j+1])
                if new_word[j] == new_token or new_word[j+1] == new_token:
                    pair_counts[pair] += freq
                    pair_to_word_indices[pair].add(i)

        pair_counts = defaultdict(int, {k: v for k, v in pair_counts.items() if v > 0})

    return vocab, merges

#### Tokenizer

In [ ]:
from __future__ import annotations

import json
import regex as re
from typing import Dict, List, Tuple, Iterable, Iterator, Any, ClassVar


PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""



def bytes_to_unicode() -> Dict[int, str]:
    bs = list(range(ord("!"), ord("~") + 1)) + \
         list(range(ord("¡"), ord("¬") + 1)) + \
         list(range(ord("®"), ord("ÿ") + 1))
    cs = bs[:]
    n = 0
    for b in range(256):
        if b not in bs:
            bs.append(b)
            cs.append(256 + n)
            n += 1
    return {b: chr(c) for b, c in zip(bs, cs)}


class Tokenizer:
    pat_str: ClassVar[str] = PAT

    def __init__(
        self,
        vocab: Dict[int, bytes],
        merges: List[Tuple[bytes, bytes]],
        special_tokens: List[str] | None = None,
    ):
        self.id_to_token = dict(vocab)
        self.token_to_id = {v: k for k, v in self.id_to_token.items()}
        self.ranks = {pair: i for i, pair in enumerate(merges)}
        
        # Pre-compile the PAT regex once
        self.compiled_pat = re.compile(self.pat_str)
        
        self.special_tokens = special_tokens or []
        if self.special_tokens:
            special_pat = "|".join(re.escape(st) for st in sorted(self.special_tokens, key=len, reverse=True))
            self.special_regex = re.compile(f"({special_pat})")
        else:
            self.special_regex = None

    @classmethod
    def from_files(cls, vocab_filepath: str, merges_filepath: str, special_tokens: List[str] | None = None) -> "Tokenizer":
        with open(vocab_filepath, "r", encoding="utf-8") as f:
            vocab_json = json.load(f)
            vocab = {int(k): v.encode("latin1") for k, v in vocab_json.items()}

        # Build the decoder dictionary ONCE here
        byte_encoder = bytes_to_unicode()
        symbol_to_byte = {v: k for k, v in byte_encoder.items()}

        merges = []
        with open(merges_filepath, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith("#version"):
                    continue
                parts = line.split()
                # Use the local decoder dictionary
                t1 = bytes([symbol_to_byte[char] for char in parts[0]])
                t2 = bytes([symbol_to_byte[char] for char in parts[1]])
                merges.append((t1, t2))

        return cls(vocab, merges, special_tokens)

    @staticmethod
    def _decode_symbol(symbol: str) -> int:
        decoder = {v: k for k, v in bytes_to_unicode().items()}
        return decoder[symbol]

    def _apply_bpe(self, token_bytes: bytes) -> List[bytes]:
        if len(token_bytes) <= 1:
            return [token_bytes]
            
        parts = [bytes([b]) for b in token_bytes]
        
        while len(parts) > 1:
            # Find all possible adjacent pairs in the current sequence
            pairs = [(parts[i], parts[i+1]) for i in range(len(parts)-1)]
            
            # Find the pair with the best (lowest) rank
            # Pairs not in ranks are ignored (float('inf'))
            best_pair = min(pairs, key=lambda p: self.ranks.get(p, float('inf')))
            
            if best_pair not in self.ranks:
                break
                
            # Perform the merge: replace all occurrences of best_pair in this sequence
            new_parts = []
            i = 0
            while i < len(parts):
                if i < len(parts) - 1 and (parts[i], parts[i+1]) == best_pair:
                    new_parts.append(parts[i] + parts[i+1])
                    i += 2
                else:
                    new_parts.append(parts[i])
                    i += 1
            parts = new_parts
        return parts

    def encode(self, text: str) -> List[int]:
        if not text:
            return []
        
        if self.special_regex:
            segments = [p for p in self.special_regex.split(text) if p]
        else:
            segments = [text]

        ids = []
        for seg in segments:
            if seg in self.special_tokens:
                ids.append(self.token_to_id[seg.encode("utf-8")])
                continue
            
            # Use the pre-compiled regex
            pieces = self.compiled_pat.findall(seg)
            
            for piece in pieces:
                piece_bytes = piece.encode("utf-8")
                for tok in self._apply_bpe(piece_bytes):
                    ids.append(self.token_to_id[tok])
        return ids

    def encode_iterable(self, iterable: Iterable[str]) -> Iterator[int]:
        for text in iterable:
            for token_id in self.encode(text):
                yield token_id

    def decode(self, ids: List[int]) -> str:
        byte_sequence = b"".join(self.id_to_token[i] for i in ids)
        return byte_sequence.decode("utf-8", errors="replace")
